# Import Libs

In [1]:
import os

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from IPython.display import Image

#sklearn
from sklearn.model_selection import train_test_split

# keras
import keras
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization

# tensorflow
import tensorflow as tf

# Load dataset

## Define Path

In [3]:
dataset_dir = './hand-sign-dataset'
train_dir = os.path.join(dataset_dir, 'asl_alphabet_train', 'asl_alphabet_train')
test_dir = os.path.join(dataset_dir, 'asl_alphabet_test', 'asl_alphabet_test')

print(train_dir)
print(test_dir)

./hand-sign-dataset/asl_alphabet_train/asl_alphabet_train
./hand-sign-dataset/asl_alphabet_test/asl_alphabet_test


## Generate Train Dataset

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(200, 200),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    seed=123,
                                                    shuffle=True)

class_names = list(train_generator.class_indices.keys())

print(class_names)

Found 87000 images belonging to 29 classes.
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


## Generate Test Dataset

In [5]:
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(200, 200),
                                                  color_mode='rgb',
                                                  batch_size=32,
                                                  class_mode='categorical',
                                                  seed=123,
                                                  shuffle=True)

Found 0 images belonging to 0 classes.


In [6]:
print(f"Images of training generator have shape: {train_generator.target_size}")

Images of training generator have shape: (200, 200)


# Create model

In [7]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200, 200, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(24, activation='softmax')])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 73728)             0         
                                                                 
 dense (Dense)               (None, 512)               3

# Train Model

In [9]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
epochs = 20
step_size_train = train_generator.n
start = time.time()

history = model.fit(train_generator,
                    # steps_per_epoch=step_size_train,
                    epochs=epochs)

end = time.time()

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Library/Frameworks/Python.framew

# Evaluate Model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

# Save Model

In [ ]:
model_path = os.path.join('./', 'saved-model', 'hand-sign-custom-model.h5')
model.save(model_path)

# Test Model

In [ ]:
path = "./for-test/letter-s.jpeg"

# display(Image(filename=path))
img = tf.keras.utils.load_img(path, color_mode ='grayscale',target_size = (28, 28))
img = tf.keras.utils.img_to_array(img)
img = np.expand_dims(img, axis = 0)
img = preprocess_input(img)
score = model.predict(img)

# print(score)
for i in range(0,len(score[0])):
    print(
    "{} with a {:.2f} percent confidence."
    .format(class_names[i], 100 * score[0][i])
    )
print(
"This image most likely belongs to {} with a {:.2f} percent confidence."
.format(class_names[np.argmax(score)], 100 * np.max(score))
)